# ML pipeline for a single assay endpoint

### Imports

In [ ]:
import os
import yaml
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, fbeta_score
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, VarianceThreshold, SelectFromModel
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier


### Load assay endpoint and QSAR-ready chemical structure fingerprints
Note: Many chemicals from assays do not have a fingerprint -> small training data

In [1]:
# Define the root directory
ROOT_DIR = os.path.dirname(os.path.abspath("__file__"))
CONFIG_PATH = os.path.join(ROOT_DIR, 'config', 'config_ml.yaml')


def load_config(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        if config["ignore_warnings"]:
            import warnings
            warnings.filterwarnings("ignore")

    return config  

 
config = load_config(CONFIG_PATH)

print(f"ML pipeline for assay ID: {config['aeid']}\n")

# Prepare assay df
assay_file = f"{config['aeid']}.csv"
assay_file_path = os.path.join(ROOT_DIR, "export", "out", assay_file)
assay_df = pd.read_csv(assay_file_path)
print(f"Assay dataframe: {assay_df.shape[0]} chemical/hitcall datapoints")
# print(f"{assay_df['hitc'].value_counts()}\n")

# Prepare fingerprint df
fingerprint_file= "ToxCast_CSIfps.csv"
fps_file_path = os.path.join(ROOT_DIR, 'input', fingerprint_file)
# Skip the first 3 columns (relativeIndex, absoluteIndex, index) and transpose the dataframe
fps_df = pd.read_csv(fps_file_path).iloc[:, 3:].T
data = fps_df.iloc[1:].values.astype(int)
index = fps_df.index[1:]
columns = fps_df.iloc[0]
fps_df = pd.DataFrame(data=data, index=index, columns=columns).reset_index()
fps_df = fps_df.rename(columns={"index": "dtxsid"})
assert fps_df.shape[0] == fps_df['dtxsid'].nunique()
print(f"Fingerprint dataframe ({fingerprint_file}): {fps_df.shape[0]} chemicals, {fps_df.iloc[:, 1:].shape[1]} binary features")

# Get intersection and merge the assay and fingerprint dataframes
df = pd.merge(assay_df, fps_df, on="dtxsid").reset_index(drop=True)
assert df.shape[0] == df['dtxsid'].nunique()
print(f"\nMerged dataframe for this ML pipeline: {df.shape[0]} datapoints (chemical fingerprint/hitcall)")


NameError: name 'os' is not defined

### Split data into train and test set

In [3]:
config = load_config(CONFIG_PATH)

# Partition the data into features (X) and labels (y)
# Select all columns as fingerprint features, starting from the third column (skipping dtxsid and hitc)
X = df.iloc[:, 2:]  
# Select the chit column (consensus hit) as the label based on the activity threshold
t = config['activity_threshold']
print(f"Activity threshold: (chit >= {t} is active)\n")
y = (df['chit'] >= t).astype(int)

# Split the data into train and test sets before oversampling to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=config['train_test_split_ratio'],
                                                    random_state=config['random_state'],
                                                    shuffle=True, # shuffle the data before splitting (default)
                                                    stratify=y) # stratify to ensure the same class distribution in the train and test sets
def print_label_count(y, title):
    counts = y.value_counts().values
    print(f"Label Count {title}: {len(y)} datapoints\n"
          f" with {counts[0]} inactive, {counts[1]} active "
          f"({counts[1]/sum(counts)*100:.2f}%)\n")

print_label_count(y, "TOTAL")
print_label_count(y_train, "TRAIN")

# Perform SMOTE oversampling? within the nested cross-validation loop with hyper-parameter tuning?
if config['apply']['smote']:
    oversampler = SMOTE(random_state=config['random_state'])
    X_train, y_train = oversampler.fit_resample(X_train, y_train)
    print_label_count(y_train, "TRAIN (after oversampling)")

print_label_count(y_test, "TEST")


Activity threshold: (chit >= 0.9 is active)

Label Count TOTAL: 7611 datapoints
 with 7296 inactive, 315 active (4.14%)

Label Count TRAIN: 6088 datapoints
 with 5836 inactive, 252 active (4.14%)

Label Count TEST: 1523 datapoints
 with 1460 inactive, 63 active (4.14%)



### Build classifier pipeline

In [4]:
config = load_config(CONFIG_PATH)

# build for each classifier a pipeline with the steps defined in the config file
def build_pipeline(steps):
    pipeline_steps = []
    for step in steps:
        step_name = step['name']  
        step_args = step.get('args', {}) # get the hyperparameters for the step, if any
        step_instance = globals()[step_name](**step_args)  # dynmically create an instance of the step
        pipeline_steps.append((step_name, step_instance))  
    return Pipeline(pipeline_steps)


def build_param_grid(classifier_steps):
    param_grid = {}
    for step in classifier_steps:
        step_name = step['name']
        step_args = step.get('args', {})
        param_grid.update({f'{step_name}__{key}': value for key, value in step_args.items() if isinstance(value, list)})
    return param_grid


def grid_search_cv(classifier, pipeline):
    scoring = config['grid_search_cv']['scoring']
    # Define the scoring function using F-beta score if specified in the config file
    scorer = make_scorer(fbeta_score, beta=config['grid_search_cv']['beta']) if scoring == 'f_beta' else scoring

    grid_search = GridSearchCV(pipeline, 
                               param_grid=build_param_grid(classifier['steps']),
                               cv=RepeatedStratifiedKFold(n_splits=5, # outer grid: cross-validation, repeated
                                                          n_repeats=3, 
                                                          random_state=config['random_state']), 
                               scoring=scorer,
                               n_jobs=config["grid_search_cv"]["n_jobs"],
                               verbose=config["grid_search_cv"]["verbose"],
                               ).fit(X_train, y_train)
    
    print(f"{classifier['name']}: GridSearchCV Results:")
    best_params = grid_search.best_params_ if grid_search.best_params_ else "default"
    print(f"Best params:\n{best_params} with mean cross-validated score: {grid_search.best_score_}\n")

    return grid_search
    

def predict_and_report(classifier, best_estimator):
    print(f"Predict..")
    y_pred = best_estimator.predict(X_test)

    labels = [True, False] 
    print(f"Classification Report {classifier['name']}:")
    print(classification_report(y_test, y_pred, labels=labels))

    cm = confusion_matrix(y_test, y_pred, labels=labels)
    tn, fp, fn, tp = cm.ravel()   # Extract values from confusion matrix
    print(f"Total: {len(y_test)} datapoints")
    print(f"Ground truth: {tn + fp} positive, {tp + fn} negative")
    print(f"Prediction: {tn + fn} positive, {tp + fp} negative")

    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Positive", "Negative"])
    cm_display.plot()
    plt.title(f"Confusion Matrix for {classifier['name']}")
    plt.show()
    

### Pipeline: Grid Search CV + Prediction + Classification report

In [5]:
config = load_config(CONFIG_PATH)

total_time_start = time.time()

for classifier in config['classifiers']:
    start_time = time.time()
    
    # Build the pipeline for the current classifier
    pipeline = build_pipeline(classifier['steps'])
    print(f"Pipeline for {classifier['name']}:\n{pipeline}\n")

    # Perform grid search on the extracted hyperparameters
    grid_search = grid_search_cv(classifier, pipeline)

    # Predict on the test set and best estimator
    predict_and_report(classifier, grid_search.best_estimator_)

    print(f"Done: {classifier['name']} >> {round(time.time() - start_time, 2)} seconds.\n{'_' * 75}\n\n")

print(f"Done. Total time >> {round(time.time() - total_time_start, 2)} seconds.\n")
    

Pipeline for RandomForestClassifier:
Pipeline(steps=[('PCA', PCA(n_components=[10])),
                ('RandomForestClassifier',
                 RandomForestClassifier(n_estimators=[100, 150]))])

Fitting 15 folds for each of 2 candidates, totalling 30 fits
